In [1]:
import sys
import math
import json
import re
import requests
import folium
import urllib.parse as encode
import shapely
import pandas as pd
import geopandas as gpd

sys.path.insert(1, '../../scripts/modules')
from DizzyAPI import DizzyAPI 
from utils import RenderJSON

### 1. Load data

In [2]:
properties = pd.read_csv('../../data/curated/curated_domain_properties.csv')
properties['geometry'] = properties.apply(lambda x: shapely.geometry.Point([x['longitude'], x['latitude']]), axis=1)
properties = gpd.GeoDataFrame(properties)
print(properties.shape)
properties.head()

(9077, 13)


,property_id,suburb,postcode,type,price_per_week,bond,bed,bath,parking,is_furnished,latitude,longitude,geometry
0,1,caulfield,3162,House,730.769231,5825.000000,0.0,0.0,0.0,0,-37.886023,145.017306,POINT (145.01731 -37.88602)
1,2,brunswick,3056,House,673.076923,3467.777778,0.0,0.0,2.0,0,-37.765592,144.963305,POINT (144.96330 -37.76559)
2,4,balwyn,3103,House,9950.000000,59700.000000,6.0,7.0,8.0,0,-37.811758,145.077551,POINT (145.07755 -37.81176)
3,5,malvern,3144,House,1154.734411,20000.000000,4.0,4.0,0.0,0,-37.848059,145.038048,POINT (145.03805 -37.84806)
4,6,melbourne,3000,Apartment / Unit / Flat,5000.000000,10000.000000,3.0,3.0,1.0,1,-37.806988,144.961014,POINT (144.96101 -37.80699)


In [3]:
sf = gpd.read_file('../../data/landing/SA2_2021_AUST_GDA2020.shp')
sf = sf[sf['STE_NAME21'] == 'Victoria']
sf = sf[['SA2_NAME21', 'AREASQKM21', 'geometry']]
sf = sf.iloc[:-2, :]
sf

,SA2_NAME21,AREASQKM21,geometry
644,Alfredton,52.7109,"POLYGON ((143.78282 -37.56666, 143.75558 -37.5..."
645,Ballarat,12.3787,"POLYGON ((143.81896 -37.55582, 143.81644 -37.5..."
646,Buninyong,51.5855,"POLYGON ((143.84171 -37.61596, 143.84176 -37.6..."
647,Delacombe,34.1607,"POLYGON ((143.75050 -37.59119, 143.75044 -37.5..."
648,Smythes Creek,104.7274,"POLYGON ((143.73296 -37.62333, 143.73263 -37.6..."
...,...,...,...
1161,Otway,1505.8895,"MULTIPOLYGON (((143.40263 -38.78152, 143.40252..."
1162,Moyne - East,3275.7989,"POLYGON ((142.41438 -38.09303, 142.41400 -38.0..."
1163,Moyne - West,2037.7626,"MULTIPOLYGON (((142.00870 -38.41715, 142.00876..."
1164,Warrnambool - North,62.4199,"POLYGON ((142.43668 -38.35544, 142.43658 -38.3..."


In [4]:
place = pd.read_csv('../../data/spatial-data/infrastructure.csv')
place['geometry'] = place.apply(lambda x: shapely.geometry.Point([x['lon'], x['lat']]), axis=1)
place = gpd.GeoDataFrame(place)
place

,name,place_type,lat,lon,geometry
0,South West Healthcare,hospital,-38.380137,142.473434,POINT (142.47343 -38.38014)
1,Northern Private Hospital,hospital,-37.653197,145.012572,POINT (145.01257 -37.65320)
2,مستشفى,hospital,-37.662012,145.052918,POINT (145.05292 -37.66201)
3,East Wimmera Health Service Donald Campus,hospital,-36.366480,142.976385,POINT (142.97639 -36.36648)
4,Hudson health care,hospital,-37.693117,144.758669,POINT (144.75867 -37.69312)
...,...,...,...,...,...
8042,Dulwich Grove,tram stop,-33.905018,151.139038,POINT (151.13904 -33.90502)
8043,Brunswick Tram Depot,tram stop,-37.757846,144.963618,POINT (144.96362 -37.75785)
8044,Spencer St/Collins St,tram stop,-37.818840,144.955084,POINT (144.95508 -37.81884)
8045,Blythe St/High St,tram stop,-37.754358,145.001235,POINT (145.00123 -37.75436)


In [5]:
schools = pd.read_csv('../../data/curated/curated_school_location.csv')
geo = []
for i in range(len(schools)):
    row = schools.iloc[i]
    geo.append(shapely.geometry.Point([row['longtitude'], row['latitude']]))
schools['geometry'] = geo
schools

,school_name,suburb,postcode,education_sector,longtitude,latitude,geometry
0,Alberton Primary School,alberton,3971,Government,146.66660,-38.61771,POINT (146.6666 -38.61771)
1,Allansford and District Primary School,allansford,3277,Government,142.59039,-38.38628,POINT (142.59039 -38.38628)
2,Avoca Primary School,avoca,3467,Government,143.47565,-37.08450,POINT (143.47565 -37.0845)
3,Avenel Primary School,avenel,3664,Government,145.23472,-36.90137,POINT (145.23472 -36.90137)
4,Warrandyte Primary School,warrandyte,3113,Government,145.21398,-37.74268,POINT (145.21398 -37.74268)
...,...,...,...,...,...,...,...
2296,Plenty River College,south morang,3752,Independent,145.08148,-37.64875,POINT (145.08148 -37.64875)
2297,Holy Cross Catholic Primary School,mickleham,3064,Catholic,144.90520,-37.53046,POINT (144.9052 -37.53046)
2298,Sidrah Gardens School,narre warren north,3804,Independent,145.31589,-37.97324,POINT (145.31589 -37.97324)
2299,Mountain District Community College,ferntree gully,3156,Independent,145.29327,-37.88319,POINT (145.29327 -37.88319)


### 2. Get distance by car

In [6]:
file stop here

SyntaxError: invalid syntax (1360562491.py, line 1)

In [46]:
GGM = DizzyAPI('../../credential/google.env', quota=370, rate=3000, cost=0.005, num_token=1)

In [47]:
def nearest_place(place_type, lat, lon):
    point = shapely.ops.nearest_points(shapely.geometry.Point([lon, lat]), place_type)[1]
    return point.y, point. x

def get_distance_by_car(lat1, lon1, lat2, lon2):
    key = GGM.get_token()
    url = 'https://maps.googleapis.com/maps/api/directions/json'\
            f'?destination={lat2},{lon2}'\
            f'&origin={lat1},{lon1}'\
            f'&units=metric'\
            f'&key={key}'
    url =  encode.quote(url, safe='/:?=&,')
    try:
        response = requests.get(url)
        return response.text
    except:
        return False

def parse_result(response):
    leg = response['routes'][0]['legs'][0]
    dis = leg['distance']['value']
    dur = leg['duration']['value']
    return dis, dur

def in_sa2(lat, lon):
    point = shapely.geometry.Point([lon, lat])
    sf['contain'] = sf.contains(point)
    if len(sf[sf['contain'] == True]) > 0:
        return sf[sf['contain'] == True].iloc[0]['SA2_NAME21'].lower()
    else:
        return 'other state' 

In [15]:
file stop here

SyntaxError: invalid syntax (1360562491.py, line 1)

In [48]:
CBD_LAT, CBD_LON = -37.81391953642295, 144.96302017517817
TRANSPORT = place[(place['place_type'] == 'train station') | (place['place_type'] == 'tram stop')].unary_union
HOSPITAL = place[place['place_type'] == 'hospital'].unary_union
POLICE = place[place['place_type'] == 'police station'].unary_union
SUPERMARKET = place[place['place_type'] == 'supermarket'].unary_union
MARKET = place[place['place_type'] == 'market'].unary_union
SHOPPING = place[place['place_type'] == 'shopping centre'].unary_union
SCHOOL = gpd.GeoDataFrame(schools).unary_union

DISTANCE = {'dist_CBD': [],
                    'dist_public_transport': [],
                    'dist_hospital': [],
                    'dist_police_station': [],
                    'dist_supermarket': [],
                    'dist_market': [],
                    'dist_shopping_center': [],
                    'dist_school': [],
                    'sa2': [],
                    'id': [],
                    'dur_CBD': [],
                    'dur_public_transport': [],
                    'dur_hospital': [],
                    'dur_police_station': [],
                    'dur_supermarket': [],
                    'dur_market': [],
                    'dur_shopping_center': [],
                    'dur_school': []}

In [50]:
def get_car_dist(lat, lon):    
    # sa2 
    try:
        DISTANCE['sa2'].append(in_sa2(lat, lon))
    except:
        DISTANCE['sa2'].append('UNKNOWN')
        
    # 1 CBD
    try:
        response = get_distance_by_car(lat, lon, CBD_LAT, CBD_LON)
        dist, dur = parse_result(json.loads(response))
        DISTANCE['dist_CBD'].append(dist)
        DISTANCE['dur_CBD'].append(dur)
    except:
        DISTANCE['dist_CBD'].append(-1)
        DISTANCE['dur_CBD'].append(-1)
        
    # 2 public transport
    try:
        place_lat, place_lon = nearest_place(TRANSPORT , lat, lon)
        response = get_distance_by_car(lat, lon, place_lat, place_lon)
        dist, dur = parse_result(json.loads(response))
        DISTANCE['dist_public_transport'].append(dist)
        DISTANCE['dur_public_transport'].append(dur)
    except:
        DISTANCE['dist_public_transport'].append(-1)
        DISTANCE['dur_public_transport'].append(-1)
    
    # 3 hospital
    try:
        place_lat, place_lon = nearest_place(HOSPITAL , lat, lon)
        response = get_distance_by_car(lat, lon, place_lat, place_lon)
        dist, dur = parse_result(json.loads(response))
        DISTANCE['dist_hospital'].append(dist)
        DISTANCE['dur_hospital'].append(dur)
    except:
        DISTANCE['dist_hospital'].append(-1)
        DISTANCE['dur_hospital'].append(-1)
    
    # 4 police
    try:
        place_lat, place_lon = nearest_place(POLICE , lat, lon)
        response = get_distance_by_car(lat, lon, place_lat, place_lon)
        dist, dur = parse_result(json.loads(response))
        DISTANCE['dist_police_station'].append(dist)
        DISTANCE['dur_police_station'].append(dur)
    except:
        DISTANCE['dist_police_station'].append(-1)
        DISTANCE['dur_police_station'].append(-1)
    
    # 5 supermarket
    try:
        place_lat, place_lon = nearest_place(SUPERMARKET , lat, lon)
        response = get_distance_by_car(lat, lon, place_lat, place_lon)
        dist, dur = parse_result(json.loads(response))
        DISTANCE['dist_supermarket'].append(dist)
        DISTANCE['dur_supermarket'].append(dur)
    except:
        DISTANCE['dist_supermarket'].append(-1)
        DISTANCE['dur_supermarket'].append(-1)
    
    # 6 market
    try:
        place_lat, place_lon = nearest_place(MARKET , lat, lon)
        response = get_distance_by_car(lat, lon, place_lat, place_lon)
        dist, dur = parse_result(json.loads(response))
        DISTANCE['dist_market'].append(dist)
        DISTANCE['dur_market'].append(dur)
    except:
        DISTANCE['dist_market'].append(-1)
        DISTANCE['dur_market'].append(-1)

    # 7 shopping
    try:
        place_lat, place_lon = nearest_place(SHOPPING , lat, lon)
        response = get_distance_by_car(lat, lon, place_lat, place_lon)
        dist, dur = parse_result(json.loads(response))
        DISTANCE['dist_shopping_center'].append(dist)
        DISTANCE['dur_shopping_center'].append(dur)
    except:
        DISTANCE['dist_shopping_center'].append(-1)
        DISTANCE['dur_shopping_center'].append(-1)
    
    # 7 school
    try:
        place_lat, place_lon = nearest_place(SCHOOL , lat, lon)
        response = get_distance_by_car(lat, lon, place_lat, place_lon)
        dist, dur = parse_result(json.loads(response))
        DISTANCE['dist_school'].append(dist)
        DISTANCE['dur_school'].append(dur)
    except:
        DISTANCE['dist_school'].append(-1)
        DISTANCE['dur_school'].append(-1)
    

In [52]:
for i in range(len(properties)):
    row = properties.iloc[i]
    print(f'{i + 1} /{len(properties)} | id: {row["property_id"]}')
    lat, lon = row['latitude'], row['longitude']
    
    # id
    DISTANCE['id'].append(row["property_id"])
    get_car_dist(lat, lon)
    pd.DataFrame(DISTANCE).to_csv('../../data/spatial-data/distance.csv')

1 /9077 | id: 1
2 /9077 | id: 2
3 /9077 | id: 4
4 /9077 | id: 5
5 /9077 | id: 6
6 /9077 | id: 7
7 /9077 | id: 8
8 /9077 | id: 9
9 /9077 | id: 11
10 /9077 | id: 12
11 /9077 | id: 13
12 /9077 | id: 14
13 /9077 | id: 15
14 /9077 | id: 16
15 /9077 | id: 17
16 /9077 | id: 18
17 /9077 | id: 19
18 /9077 | id: 20
19 /9077 | id: 21
20 /9077 | id: 22
21 /9077 | id: 23
22 /9077 | id: 24
23 /9077 | id: 25
24 /9077 | id: 26
25 /9077 | id: 27
26 /9077 | id: 28
27 /9077 | id: 29
28 /9077 | id: 30
29 /9077 | id: 31
30 /9077 | id: 32
31 /9077 | id: 33
32 /9077 | id: 34
33 /9077 | id: 35
34 /9077 | id: 36
35 /9077 | id: 37
36 /9077 | id: 38
37 /9077 | id: 39
38 /9077 | id: 40
39 /9077 | id: 41
40 /9077 | id: 42
41 /9077 | id: 43
42 /9077 | id: 44
43 /9077 | id: 45
44 /9077 | id: 46
45 /9077 | id: 47
46 /9077 | id: 48
47 /9077 | id: 49
48 /9077 | id: 50
49 /9077 | id: 51
50 /9077 | id: 52
51 /9077 | id: 53
52 /9077 | id: 54
53 /9077 | id: 55
54 /9077 | id: 56
55 /9077 | id: 57
56 /9077 | id: 58
57 /9077 

### 4. Save data

In [60]:
dist_df = pd.DataFrame(DISTANCE)
col = ['id', 'sa2'] + [item for item in dist_df.columns if (item != 'id') and (item != 'sa2')]
dist_df =dist_df[col]
dist_df

,id,sa2,dist_CBD,dist_public_transport,dist_hospital,dist_police_station,dist_supermarket,dist_market,dist_shopping_center,dist_school,dur_CBD,dur_public_transport,dur_hospital,dur_police_station,dur_supermarket,dur_market,dur_shopping_center,dur_school
0,1,caulfield - south,12139,614,617,1653,1030,410,2080,1021,1567,146,175,246,202,110,335,213
1,2,brunswick - north,5580,940,1767,387,4141,516,1393,847,1174,177,376,83,668,105,319,203
2,4,balwyn,15333,2821,2322,3863,3556,3664,3799,754,1679,371,275,503,477,523,567,162
3,5,malvern - glen iris,9793,572,2439,1563,1974,1054,1337,1263,1116,107,441,255,341,236,229,289
4,6,melbourne cbd - north,897,575,1071,1587,390,1407,582,678,336,232,292,395,159,327,234,161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9072,9132,benalla,214599,535,1947,3026,1200,132,2249,1464,8632,49,190,285,104,11,275,131
9073,9133,myrtleford,290411,6272,8020,7430,7180,7846,34194,8153,11639,352,527,451,420,486,1685,548
9074,9134,kyabram,212892,1291,2232,1477,2640,1399,32181,1437,8801,122,192,171,266,125,1481,162
9075,9135,kyabram,227625,9832,15381,1444,1243,1243,25449,1517,9388,569,751,159,181,181,1187,172


In [61]:
dist_df.to_csv('../../data/spatial-data/distance_duration.csv', index=None)